In [211]:
import robotoc
from robotoc_sim import MPCSimulation, CameraSettings
import numpy as np
import roboticstoolbox as rtb
from roboticstoolbox import models
from roboticstoolbox.robot.Robot import Robot
from spatialmath import SE3



In [212]:

model_info = robotoc.RobotModelInfo()
model_info.urdf_path = '../a1_description/urdf/a1.urdf'
model_info.base_joint_type = robotoc.BaseJointType.FloatingBase
baumgarte_time_step = 0.05
model_info.point_contacts = [robotoc.ContactModelInfo('FL_foot', baumgarte_time_step),
                             robotoc.ContactModelInfo('RL_foot', baumgarte_time_step),
                             robotoc.ContactModelInfo('FR_foot', baumgarte_time_step),
                             robotoc.ContactModelInfo('RR_foot', baumgarte_time_step)]


In [213]:
from roboticstoolbox.robot.ERobot import ERobot


class A1(ERobot):
        """
        Class that imports a URDF model
        """

        def __init__(self, urdf_file):

            links, name, urdf_string, urdf_filepath = self.URDF_read(urdf_file)

            super().__init__(
                links,
                name=name,
                manufacturer="Unitree",
                urdf_string=urdf_string,
                urdf_filepath=urdf_filepath,
            )


In [214]:
rtb_a1robot = A1("/home/sharath/robotoc/examples/a1/a1_description/urdf/a1.urdf")

In [215]:
rtb_a1robot

ERobot: a1_description (by Unitree), 12 joints (RRRRRRRRRRRR), 9 branches, dynamics, geometry, collision
┌─────┬────────────────────┬───────┬──────────┬──────────────────────────────────┐
│link │        link        │ joint │  parent  │       ETS: parent to link        │
├─────┼────────────────────┼───────┼──────────┼──────────────────────────────────┤
│   0 │ base               │       │ BASE     │ SE3()                            │
│   1 │ trunk              │       │ base     │ SE3()                            │
│   2 │ @imu_link          │       │ trunk    │ SE3()                            │
│   3 │ FR_hip             │     0 │ trunk    │ SE3(0.1805, -0.047, 0) ⊕ Rx(q0)  │
│   4 │ @FR_thigh_shoulder │       │ FR_hip   │ SE3(0, -0.081, 0)                │
│   5 │ FR_thigh           │     1 │ FR_hip   │ SE3(0, -0.0838, 0) ⊕ Ry(q1)      │
│   6 │ FR_calf            │     2 │ FR_thigh │ SE3(0, 0, -0.2) ⊕ Ry(q2)         │
│   7 │ @FR_foot           │       │ FR_calf  │ SE3(0, 0, -0.2)  

In [223]:
T = SE3([
    [0.808,  0,  -0.5891, -0.1889],
    [0,      1,   0,      -0.209],
    [0.5891, 0,   0.808,  -0.3184],
    [0,      0,   0,       1]
])

In [224]:
rtb_a1robot.ikine_LM(T,start=rtb_a1robot.links[0],end=rtb_a1robot.links[17],mask = [1/np.sqrt(3),1/np.sqrt(3),1/np.sqrt(3),0,0,0])

IKSolution(q=array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.23387746,  0.54292848, -1.03748979]), success=False, iterations=836, searches=100, residual=1.8851017356552417e-10, reason='iteration and search limit reached, solution found but violates joint limits')

In [225]:
robot = robotoc.Robot(model_info)
t0 = 0.0
q0 = np.array([0, 0, 0.3181, 0, 0, 0, 1, #CoM
               -0.0916,  0.6332,  -1.3497, #LF
               -0.3197,  0.3287,  -0.8457, #RF
               -0.1360,  0.5179, -1.340, #LH
               -0.2338,  0.5429,  -1.037]) #RH
               
robot.forward_kinematics(q0)
com_ref0 = robot.com()
x3d0_LF = robot.frame_position('FL_foot')
x3d0_LH = robot.frame_position('RL_foot')
x3d0_RF = robot.frame_position('FR_foot')
x3d0_RH = robot.frame_position('RR_foot')

In [230]:
x3d0_LH

array([-0.13299595,  0.08800839, -0.00030908])